# Commodity Sales Dashboard
<https://public.tableau.com/app/profile/sieger.bokschoten/viz/USDACensusandAgricultureLabor/CommoditySalesDashboard>

The only dependency required is pandas which can be installed with pip install pandas, or in a conda environment conda install pandas.

The data source is the NASS USDA 2022 Census, which can be downloaded here: https://www.nass.usda.gov/datasets/qs.census2022.txt.gz
The file will need to be unzipped and placed in the folder you will be using for your project.

The census text file is a tab-separated text file, and uncompressed is around 2.2 GB. It's normal for the first command below to take 20 seconds or more to load the full dataset to memory.

In this document I'm not going very in depth to how to decide which columns to filter, or what values will be contained in those columns, the reason is because before developing this documentation, I was exploring the data using the Spyder IDE. It has a useful variable explorer and IPython built in. If you want to explore the data yourself, I recommend using an IDE to walk through these steps and view the output. I'm skipping the data exploration steps because the full process is longer and documenting the process in Jupyter Notebooks is time consuming.

In [1]:
import pandas as pd

df = pd.read_table('qs.census2022.txt', sep='\t')

Next, we'll take a quick look at the data types to make sure the dataframe was loaded.

In [7]:
print(df.dtypes)

SOURCE_DESC               object
SECTOR_DESC               object
GROUP_DESC                object
COMMODITY_DESC            object
CLASS_DESC                object
PRODN_PRACTICE_DESC       object
UTIL_PRACTICE_DESC        object
STATISTICCAT_DESC         object
UNIT_DESC                 object
SHORT_DESC                object
DOMAIN_DESC               object
DOMAINCAT_DESC            object
AGG_LEVEL_DESC            object
STATE_ANSI               float64
STATE_FIPS_CODE            int64
STATE_ALPHA               object
STATE_NAME                object
ASD_CODE                 float64
ASD_DESC                  object
COUNTY_ANSI              float64
COUNTY_CODE              float64
COUNTY_NAME               object
REGION_DESC              float64
ZIP_5                    float64
WATERSHED_CODE             int64
WATERSHED_DESC           float64
CONGR_DISTRICT_CODE      float64
COUNTRY_CODE               int64
COUNTRY_NAME              object
LOCATION_DESC             object
YEAR      

Now that we have the dataframe loaded we will need to start filtering rows so we reduce the size of the dataset.

In [2]:
sales = df[df['SECTOR_DESC'] == 'CROPS']
sales = sales[sales['STATISTICCAT_DESC'] == 'SALES']
sales = sales[sales['UNIT_DESC'] == '$']
sales = sales[sales['DOMAIN_DESC'] == 'TOTAL']

Now we need to remove all rows that are not counties, otherwise the data will be strongly skewed because the dataset is currently including US totals and state totals. Then we'll sort the data by state and county names and reindex. On the rows where STATE_NAME is nan, the total will refer to the US total. On the COUNTY_NAME rows where county is nan, the total will refer to either the state total, or the US total. We can remove both state totals, and US totals by only returning rows where COUNTY_NAME is not NaN.

In [3]:
sales = sales[sales['COUNTY_NAME'].notna()]
sales.sort_values(by=['STATE_NAME', 'COUNTY_NAME'], inplace=True)
sales = sales.reset_index(drop=True)

The next step is to change the 'VALUE' column datatype to Int64. Currently it's an object in Pandas, or a string with other software. This column currently contains comma seperated numbers to represent dollars in sales, and it contains the values '(D)' and '(Z)' which represent values that cannot be reported. Beacause we can't compare the values of columns with '(D)' or '(Z)', and those are not equal to zero, we'll convert them to NaN, so they don't affect any calculations. If you were to add these numbers up and compare to state totals, it's likely that they won't match up correctly, this is most likely because the USDA has access to the '(D)' or '(Z)' values, and so can add them to the final state totals.

In [4]:
sales['VALUE'] = sales['VALUE'].str.replace(',', '')
sales['VALUE'] = pd.to_numeric(sales['VALUE'], errors='coerce')
sales['VALUE'] = sales['VALUE'].astype('Int64')

We can verify that the column was converted succesfully by running the below command.

In [5]:
sales['VALUE'].dtype

Int64Dtype()

The final steps are to remove unused columns, keeping only those that we may use for visualizations, then we export the file as a csv.

In [6]:
sales = sales[['STATE_NAME', 'STATE_ALPHA', 'COUNTY_NAME', 'GROUP_DESC', 'COMMODITY_DESC', 'SHORT_DESC', 'VALUE']]

In [7]:
print(sales.dtypes)

STATE_NAME        object
STATE_ALPHA       object
COUNTY_NAME       object
GROUP_DESC        object
COMMODITY_DESC    object
SHORT_DESC        object
VALUE              Int64
dtype: object


In [8]:
sales.to_csv('Commodity_Sales.csv')